In [13]:
import dai
import pandas as pd
import numpy as np
import time
from datetime import timedelta

In [14]:
ed = dai.query("SELECT DISTINCT date FROM cn_stock_bar1d WHERE date >= '2024-01-01'").df()['date'].max().strftime("%Y-%m-%d")
sd = (pd.Timestamp(ed) - timedelta(days=45)).strftime("%Y-%m-%d")

In [15]:
def get_factor_df_one_instrument_one_day(instrument, date):

    sql = f"""
    WITH 

    data1 AS (
        SELECT
            date AS date_time,
            make_date(year(date_time),month(date_time),day(date_time)) AS date,
            instrument,
            bs_flag,
            price,
            volume,
            CASE WHEN ask_seq_num IN (SELECT ask_seq_num FROM cn_stock_level2_trade GROUP BY trading_day, instrument, ask_seq_num HAVING COUNT(*) > 1) THEN 83 ELSE 66 END AS bs2,
        FROM cn_stock_level2_trade
        WHERE date >= '{date} 00:00:00.000' AND date <= '{date} 23:59:59.999'
        AND instrument = '{instrument}'
    ),

    data2 AS (
        SELECT
            date,
            instrument,
            CONCAT(CAST(year(date) AS VARCHAR), CAST(week(date) AS VARCHAR))  AS week,
            CONCAT(CAST(year(date) AS VARCHAR), CAST(month(date) AS VARCHAR)) AS month,
            close / adjust_factor  AS close,
        FROM cn_stock_bar1d
        WHERE date >= '{date} 00:00:00.000' AND date <= '{date} 23:59:59.999'
        AND instrument = '{instrument}'
    ),

    data3 AS (

        SELECT
            
            data2.date AS date,
            data2.instrument AS instrument,
            data2.week AS _week,
            data2.month AS _month,

            data1.date_time AS _date_time,
            data1.bs_flag AS _bs_flag,
            data1.price AS _price,
            data2.close AS _close,
            data1.volume AS _volume,
            data1.bs2 AS _bs2,

            IF(make_time(hour(_date_time), minute(_date_time), second(_date_time)) >= '14:30:00' AND make_time(hour(_date_time), minute(_date_time), second(_date_time)) <= '14:57:00', 1, 0) AS _is_market_close,
            SUM(_volume) OVER(PARTITION BY date, instrument) AS _total_volume,
            SUM(_volume * _is_market_close) OVER(PARTITION BY date, instrument) AS _total_volume_market_close,
            AVG(_volume) OVER(PARTITION BY date, instrument) AS _mean_volume,

            IF(_bs_flag = 66, 1, 0) AS _is_buy_order,
            IF(_bs_flag = 83, 1, 0) AS _is_sell_order,
            IF(_bs2 = 66, 1, 0) AS _is_buy_order_2,
            IF(_bs2 = 83, 1, 0) AS _is_sell_order_2,
            IF(_price < _close, 1, 0) AS _is_small_price,
            IF(_price > _close, 1, 0) AS _is_large_price,
            IF(_volume < _mean_volume, 1, 0) AS _is_small_volume,
            IF(_volume > _mean_volume, 1, 0) AS _is_large_volume,
            
            -- Factor Calculation
            -- 1. Daily
            -- 01 HCVOL_D
            SUM(_volume * _is_buy_order    * _is_large_price) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOL_D,

            -- 02 LCVOL_D
            SUM(_volume * _is_sell_order   * _is_small_price) OVER(PARTITION BY date, instrument) / _total_volume 
            AS LCVOL_D,

            -- 03 HCP_D
            IF(
                SUM(_is_buy_order * _is_large_price) OVER(PARTITION BY date, instrument) = 0, 
                0, 
                SUM(_price * _is_buy_order    * _is_large_price) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order * _is_large_price) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCP_D,

            -- 04 LCP_D
            IF(
                SUM(_is_sell_order * _is_small_price) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price * _is_sell_order   * _is_small_price) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order * _is_small_price) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCP_D,


            -- 05 HCVOLS_D
            SUM(_volume * _is_buy_order_2  * _is_large_price * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOLS_D,

            -- 06 LCVOLS_D
            SUM(_volume * _is_sell_order_2 * _is_small_price * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS LCVOLS_D,

            -- 07 HCPS_D
            IF(
                SUM(_is_buy_order_2  * _is_large_price * _is_small_volume) OVER(PARTITION BY date, instrument) = 0, 
                0,
                SUM(_price * _is_buy_order_2  * _is_large_price * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order_2  * _is_large_price * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCPS_D,

            -- 08 LCPS_D
            IF(
                SUM(_is_sell_order_2 * _is_small_price * _is_small_volume) OVER(PARTITION BY date, instrument) = 0, 
                0,
                SUM(_price * _is_sell_order_2 * _is_small_price * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order_2 * _is_small_price * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCPS_D,

            -- 09 HCVOLE1_D
            SUM(_volume * _is_buy_order * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOLE1_D,

            -- 10 HCVOLE2_D
            SUM(_volume * _is_buy_order * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS HCVOLE2_D,

            -- 11 HCVOLE3_D
            SUM(_volume * _is_buy_order_2 * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOLE3_D,

            -- 12 HCVOLE4_D
            SUM(_volume * _is_buy_order_2 * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS HCVOLE4_D,

            -- 13 LCVOLE1_D
            SUM(_volume * _is_sell_order * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume              
            AS LCVOLE1_D,

            -- 14 LCVOLE2_D
            SUM(_volume * _is_sell_order * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS LCVOLE2_D,

            -- 15 LCVOLE3_D
            SUM(_volume * _is_sell_order_2 * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume              
            AS LCVOLE3_D,

            -- 16 LCVOLE4_D
            SUM(_volume * _is_sell_order_2 * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS LCVOLE4_D,

            -- 17 HCPE1_D
            IF(
                SUM(_is_buy_order    * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_buy_order    * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order    * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCPE1_D,

            -- 18 HCPE2_D
            IF(
                SUM(_is_buy_order_2  * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_buy_order_2  * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order_2  * _is_large_price * _is_market_close) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCPE2_D,

            -- 19 LCPE1_D
            IF(
                SUM(_is_sell_order   * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_sell_order   * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order   * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCPE1_D,

            -- 20 LCPE2_D
            IF(
                SUM(_is_sell_order_2 * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_sell_order_2 * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order_2 * _is_small_price * _is_market_close) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCPE2_D,

            -- 21 HCVOLES1_D
            SUM(_volume * _is_buy_order * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOLES1_D,

            -- 22 HCVOLES2_D
            SUM(_volume * _is_buy_order * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS HCVOLES2_D,

            -- 23 HCVOLES3_D
            SUM(_volume * _is_buy_order_2 * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS HCVOLES3_D,

            -- 24 HCVOLES4_D
            SUM(_volume * _is_buy_order_2 * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS HCVOLES4_D,

            -- 25 LCVOLES1_D
            SUM(_volume * _is_sell_order * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS LCVOLES1_D,

            -- 26 LCVOLES2_D
            SUM(_volume * _is_sell_order * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS LCVOLES2_D,

            -- 27 LCVOLES3_D
            SUM(_volume * _is_sell_order_2 * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume 
            AS LCVOLES3_D,

            -- 28 LCVOLES4_D
            SUM(_volume * _is_sell_order_2 * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _total_volume_market_close 
            AS LCVOLES4_D,

            -- 29 HCPES1_D
            IF(
                SUM(_is_buy_order    * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_buy_order    * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order    * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCPES1_D,
            
            -- 30 HCPES2_D
            IF(
                SUM(_is_buy_order_2  * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_buy_order_2  * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_buy_order_2  * _is_large_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS HCPES2_D,
            
            -- 31 LCPES1_D
            IF(
                SUM(_is_sell_order   * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_sell_order   * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order   * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCPES1_D,
            
            -- 32 LCPES2_D
            IF(
                SUM(_is_sell_order_2 * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) = 0,
                0,
                SUM(_price  * _is_sell_order_2 * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / SUM(_is_sell_order_2 * _is_small_price * _is_market_close * _is_small_volume) OVER(PARTITION BY date, instrument) / _close - 1
            ) 
            AS LCPES2_D,

        FROM data1 JOIN data2 USING (date, instrument)
        ORDER BY date, instrument

    ),

    data4 AS (
        SELECT DISTINCT * 
        FROM data3 
        ORDER BY date, instrument
    ),

    data5 AS (
        SELECT
            CAST(date AS TIMESTAMP) AS date,
            instrument,
            HCVOL_D    AS alpha_hf_00040001,
            LCVOL_D    AS alpha_hf_00040002,
            HCP_D      AS alpha_hf_00040003,
            LCP_D      AS alpha_hf_00040004,
            HCVOLS_D   AS alpha_hf_00040005,
            LCVOLS_D   AS alpha_hf_00040006,
            HCPS_D     AS alpha_hf_00040007,
            LCPS_D     AS alpha_hf_00040008,
            HCVOLE1_D  AS alpha_hf_00040009,
            HCVOLE2_D  AS alpha_hf_00040010,
            HCVOLE3_D  AS alpha_hf_00040011,
            HCVOLE4_D  AS alpha_hf_00040012,
            LCVOLE1_D  AS alpha_hf_00040013,
            LCVOLE2_D  AS alpha_hf_00040014,
            LCVOLE3_D  AS alpha_hf_00040015,
            LCVOLE4_D  AS alpha_hf_00040016,
            HCPE1_D    AS alpha_hf_00040017,
            HCPE2_D    AS alpha_hf_00040018,
            LCPE1_D    AS alpha_hf_00040019,
            LCPE2_D    AS alpha_hf_00040020,
            HCVOLES1_D AS alpha_hf_00040021,
            HCVOLES2_D AS alpha_hf_00040022,
            HCVOLES3_D AS alpha_hf_00040023,
            HCVOLES4_D AS alpha_hf_00040024,
            LCVOLES1_D AS alpha_hf_00040025,
            LCVOLES2_D AS alpha_hf_00040026,
            LCVOLES3_D AS alpha_hf_00040027,
            LCVOLES4_D AS alpha_hf_00040028,
            HCPES1_D   AS alpha_hf_00040029,
            HCPES2_D   AS alpha_hf_00040030,
            LCPES1_D   AS alpha_hf_00040031,
            LCPES2_D   AS alpha_hf_00040032,
        FROM data4
    )

    SELECT * 
    FROM data5
    ORDER BY date, instrument
    """
    
    data = dai.query(sql, filters = {'date':[f"{date} 00:00:00.000", f"{date} 23:59:59.999"], "instrument":[instrument]}).df()
    return data 

In [ ]:
get_factor_df_one_instrument_one_day('600519.SH', ed)